# BDH Phased Migration Approach

The notebook will show how to demonstrate how a phased migration approach of trades mastered in portfolios in LoanIQ and Murex with different data models in separate phases as follows:

Phase 1: Data onboarded into LUSID using the existing source system data models, and simple LUISD Instruments

Phase 2: Data updated so that Instrument economics are understood by LUISD, allowing LUSID features to be used, but retaining the existing source system data model

Phase 3: Migration to a canonical data model, referred to here as the SDM (Santander Data Model)

**Table of Contents:**
- [1. Create Custom Properties](#1.-Create-Custom-Properties)
- [2. Create Portfolios](#2.-Create-Portfolios)
- [3. Phase I: Data load](#3.-Phase-I:-Data-load)
- [4. Phase II: Data load](#4.-Phase-II:-Data-load)
- [5. Phase III: Data load](#5.-Phase-III:-Data-load)
- [6. Load FX Market Data](#6.-Load-FX-Market-Data)
- [6. Valuation](#6.-Valuation)
- [7. Trade Lifecycle](#7.-Trade-Lifecycle)

In [1]:
# Import generic non-LUSID packages
import os
import pandas as pd
from datetime import datetime, timedelta
import re
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import json
import pytz
import numpy as np
from IPython.core.display import HTML

# Import key modules from the LUSID package
import lusid
import lusid.models as lm
import lusid.api as la
from lusid.utilities import ApiClientFactory
from lusidtools.pandas_utils.lusid_pandas import lusid_response_to_data_frame
from lusidtools.cocoon.transaction_type_upload import upsert_transaction_type_alias

# Import key functions from Lusid-Python-Tools and other packages
from lusidjam import RefreshingToken

# Set DataFrame display formats
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.options.display.float_format = "{:,.4f}".format
display(HTML("<style>.container { width:90% !important; }</style>"))

# Set the secrets path
secrets_path = os.getenv("FBN_SECRETS_PATH")

# For running the notebook locally
if secrets_path is None:
    secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")

# Authenticate our user and create our API client
api_factory = ApiClientFactory(
    token=RefreshingToken(),
    api_secrets_filename = secrets_path)

print ('LUSID Environment Initialised')
print ('LUSID API Version :', api_factory.build(lusid.api.ApplicationMetadataApi).get_lusid_versions().build_version)

LUSID Environment Initialised
LUSID API Version : 0.6.9978.0


In [2]:
# Set required APIs
portfolio_api = api_factory.build(lusid.api.PortfoliosApi)
transaction_portfolios_api = api_factory.build(lusid.api.TransactionPortfoliosApi)
instruments_api = api_factory.build(lusid.api.InstrumentsApi)
quotes_api = api_factory.build(lusid.api.QuotesApi)
configuration_recipe_api = api_factory.build(lusid.api.ConfigurationRecipeApi)
complex_market_data_api = api_factory.build(lusid.api.ComplexMarketDataApi)
aggregation_api = api_factory.build(lusid.api.AggregationApi)
reconciliation_api = api_factory.build(lusid.api.ReconciliationsApi)
property_definition_api = api_factory.build(la.PropertyDefinitionsApi)
portfolio_groups_api = api_factory.build(la.PortfolioGroupsApi)

In [3]:
# Define scopes
loanIQ_scope = "bdh-loanIQ"
murex_scope = "bdh-murex"
sdm_scope = "bdh-sdm"
market_data_scope = "bdh"
market_supplier = "Lusid"

# 1. Create Custom Properties

In this section we load the example trade data sets from Murex and LoanIQ, and create custom properties in LUSID to represent the Murex, LoanIQ and SDM data models.

In [4]:
def create_custom_property (domain, property_scope, code, display_name, data_type_scope, data_type_code): 
    # Create a property definition with a unique scope and code
    property_definition = lm.CreatePropertyDefinitionRequest(
        domain = domain,
        scope = property_scope,
        code = code,
        display_name = display_name,
        data_type_id = lm.ResourceId(
            scope = data_type_scope,
            code = data_type_code
        )
    )

    # Upsert property definition to LUSID
    try:
        upsert_property_definition_response = property_definition_api.create_property_definition(
            create_property_definition_request = property_definition
        )
        print(f"Property definition created with the following key: {upsert_property_definition_response.key}")
    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
                print(f"Property definition with the following key already exists: {property_definition.domain}/{property_definition.scope}/{property_definition.code}")
    return property_definition

In [5]:
def create_derived_property(domain, property_scope, code, display_name, data_type_scope, data_type_code, derivation_formula):
    derived_property_definition = lm.CreateDerivedPropertyDefinitionRequest(
        domain = domain,
        scope = property_scope,
        code = code,
        display_name = display_name,
        data_type_id = lm.ResourceId(
            scope = data_type_scope,
            code = data_type_code
        ),
        derivation_formula = derivation_formula
    )
    # Upsert property definition to LUSID
    try:
        upsert_property_definition_response = property_definition_api.create_derived_property_definition(
            create_derived_property_definition_request = derived_property_definition
        )
        print(f"Property definition created with the following key: {upsert_property_definition_response.key}")
    except lusid.ApiException as e:
        if json.loads(e.body)["name"] == "PropertyAlreadyExists":
            print(f"Property definition with the following key already exists: {derived_property_definition.domain}/{derived_property_definition.scope}/{derived_property_definition.code}")
    return derived_property_definition

In [6]:
# function to tidy up old property definitions
def delete_custom_properties(domain, scope):
    # Obtain the LUSID Property Definition API
    property_definition_api = api_factory.build(la.PropertyDefinitionsApi)

    search_api = api_factory.build(la.SearchApi)
    
    property_defs = search_api.search_properties()
     
    #display(property_defs.values)
    
    for property_def in property_defs.values:
        if property_def.domain == domain and property_def.scope == scope:
            property_definition_api.delete_property_definition(
                domain = domain,
                scope = scope,
                code = property_def.code
            )
            print(f"Deleted code: {property_def.code}")

In [7]:
delete_custom_properties ("Transaction", loanIQ_scope)
delete_custom_properties ("Transaction", murex_scope)
delete_custom_properties ("Transaction", sdm_scope)

Deleted code: instrument_type
Deleted code: strategy
Deleted code: instrument_type
Deleted code: strategy
Deleted code: strategy
Deleted code: instrument_type
Deleted code: strategy
Deleted code: executor
Deleted code: asset_name
Deleted code: exposure
Deleted code: rating
Deleted code: identifier
Deleted code: average_retail_lease
Deleted code: freq
Deleted code: term
Deleted code: no_units
Deleted code: RelatedFinancialCenter
Deleted code: ISOCurrencyCode
Deleted code: CenterID
Deleted code: Currency
Deleted code: CashType
Deleted code: exchange_rate
Deleted code: holding_type
Deleted code: StockLendingStatus
Deleted code: StockLendingStatus
Deleted code: CountryCode
Deleted code: ExerciseDate
Deleted code: ReplacementCost
Deleted code: ParValue
Deleted code: ConversionRatio
Deleted code: GICSIndustry
Deleted code: ExchangeCode
Deleted code: SecurityType
Deleted code: GICSSector
Deleted code: PaymentFrequency
Deleted code: CounterParty
Deleted code: Coupon
Deleted code: ResultingStoc

In [8]:
# search for property defs
# search_api = api_factory.build(la.SearchApi)
# property_defs = search_api.search_properties()
# display(property_defs.values)

In [9]:
def add_property_keys(property_scope, name, propertyType, trade_header):
    property_keys = []

    for index, row in trade_header.iterrows():

        # Create Custom property
        property_definition = create_custom_property ("Transaction", property_scope, row[name], row[name], "system", row[propertyType])
        property_key = f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"
        property_keys.append(property_key)

    return property_keys

In [10]:
def add_derived_property_keys(property_scope, name, propertyType, derived_name, trade_header):
    property_keys = []

    for index, row in trade_header.iterrows():

        # Create Custom property
        property_definition = create_derived_property ("Transaction", property_scope, row[name], row[name], "system", row[propertyType], f"Properties[Transaction/{property_scope}/{row[derived_name]}]")
        property_key = f"{property_definition.domain}/{property_definition.scope}/{property_definition.code}"
        property_keys.append(property_key)

    return property_keys

## 1.1 Load Murex properties 

In [11]:
# Read Murex mapping data
murex_header_df = pd.read_csv("data/Murex-mapping.csv")
murex_header_df.head()

,Name,Type,SimpleInstMap,FullInstMap,SDMMapping
0,dealReference,string,clientInternal,clientInternal,tradeReference
1,description,string,NaN,NaN,tradeDescription
2,dealGroup,string,NaN,NaN,tradeGroup
3,account,string,NaN,NaN,accountReference
4,product,string,product,product,productType


In [12]:
# These are the Phase 1 and Phase 2 property Keys (it uses the Murex default model)
property_keys = add_property_keys (murex_scope, "Name", "Type", murex_header_df)
murex_header_df['Key'] = property_keys

# These are the Phase 3 property keys
property_keys = add_property_keys (sdm_scope, "SDMMapping", "Type", murex_header_df)
murex_header_df['SDMKey'] = property_keys

murex_header_df.head(5)

Property definition created with the following key: Transaction/bdh-murex/dealReference
Property definition created with the following key: Transaction/bdh-murex/description
Property definition created with the following key: Transaction/bdh-murex/dealGroup
Property definition created with the following key: Transaction/bdh-murex/account
Property definition created with the following key: Transaction/bdh-murex/product
Property definition created with the following key: Transaction/bdh-murex/portfolio
Property definition created with the following key: Transaction/bdh-murex/desk
Property definition created with the following key: Transaction/bdh-murex/partyReference
Property definition created with the following key: Transaction/bdh-murex/tradeDate
Property definition created with the following key: Transaction/bdh-murex/maturityDate
Property definition created with the following key: Transaction/bdh-murex/expiryDate
Property definition created with the following key: Transaction/bdh-mu

,Name,Type,SimpleInstMap,FullInstMap,SDMMapping,Key,SDMKey
0,dealReference,string,clientInternal,clientInternal,tradeReference,Transaction/bdh-murex/dealReference,Transaction/bdh-sdm/tradeReference
1,description,string,NaN,NaN,tradeDescription,Transaction/bdh-murex/description,Transaction/bdh-sdm/tradeDescription
2,dealGroup,string,NaN,NaN,tradeGroup,Transaction/bdh-murex/dealGroup,Transaction/bdh-sdm/tradeGroup
3,account,string,NaN,NaN,accountReference,Transaction/bdh-murex/account,Transaction/bdh-sdm/accountReference
4,product,string,product,product,productType,Transaction/bdh-murex/product,Transaction/bdh-sdm/productType


## 1.2 Load LoanIQ properties

In [13]:
# Read LoanIQ mapping data
loanIQ_header_df = pd.read_csv("data/LoanIQ-mapping.csv")
loanIQ_header_df.head()

,Name,Type,SimpleInstMap,FullInstMap,SDMMapping
0,balanceSheetDate,date,NaN,NaN,balanceSheetDate
1,contractReference,string,clientInternal,clientInternal,tradeReference
2,contractDescription,string,NaN,NaN,tradeDescription
3,contractGroup,string,NaN,NaN,tradeGroup
4,accountReference,string,NaN,NaN,accountReference


In [14]:
# These are the Phase 1 and Phase 2 property Keys (it uses the Loan IQ default model)
property_keys = add_property_keys (loanIQ_scope, "Name", "Type", loanIQ_header_df)
loanIQ_header_df['Key'] = property_keys

# These are the Phase 3 property keys
property_keys = add_property_keys (sdm_scope, "SDMMapping", "Type", loanIQ_header_df)
loanIQ_header_df['SDMKey'] = property_keys

loanIQ_header_df.head(5)

Property definition created with the following key: Transaction/bdh-loanIQ/balanceSheetDate
Property definition created with the following key: Transaction/bdh-loanIQ/contractReference
Property definition created with the following key: Transaction/bdh-loanIQ/contractDescription
Property definition created with the following key: Transaction/bdh-loanIQ/contractGroup
Property definition created with the following key: Transaction/bdh-loanIQ/accountReference
Property definition created with the following key: Transaction/bdh-loanIQ/productType
Property definition created with the following key: Transaction/bdh-loanIQ/book
Property definition created with the following key: Transaction/bdh-loanIQ/signatureDate
Property definition created with the following key: Transaction/bdh-loanIQ/originDate
Property definition created with the following key: Transaction/bdh-loanIQ/maturityDate
Property definition created with the following key: Transaction/bdh-loanIQ/currency
Property definition creat

,Name,Type,SimpleInstMap,FullInstMap,SDMMapping,Key,SDMKey
0,balanceSheetDate,date,NaN,NaN,balanceSheetDate,Transaction/bdh-loanIQ/balanceSheetDate,Transaction/bdh-sdm/balanceSheetDate
1,contractReference,string,clientInternal,clientInternal,tradeReference,Transaction/bdh-loanIQ/contractReference,Transaction/bdh-sdm/tradeReference
2,contractDescription,string,NaN,NaN,tradeDescription,Transaction/bdh-loanIQ/contractDescription,Transaction/bdh-sdm/tradeDescription
3,contractGroup,string,NaN,NaN,tradeGroup,Transaction/bdh-loanIQ/contractGroup,Transaction/bdh-sdm/tradeGroup
4,accountReference,string,NaN,NaN,accountReference,Transaction/bdh-loanIQ/accountReference,Transaction/bdh-sdm/accountReference


# 2. Create Portfolios

We create 6 portfolios in total, one for LoanIQ and Murex for each stage. The portfolio currency is EUR in all cases.

In [15]:
def create_portfolio(portfolio_name, portfolio_scope, instrument_scope):

    try:
        transaction_portfolios_api.create_portfolio(
            scope=portfolio_scope,
            create_transaction_portfolio_request=lm.CreateTransactionPortfolioRequest(
                display_name=portfolio_name,
                code=portfolio_name,
                base_currency="EUR",
                created="2010-01-01",
                sub_holding_keys = [],
                instrument_scopes=[instrument_scope]
            ),
        )

    except lusid.ApiException as e:
        print(e.body)

In [16]:
ph1_murex_portfolio = "Ph1_Murex"
ph2_murex_portfolio = "Ph2_Murex"
ph3_murex_portfolio = "Ph3_Murex"

ph1_loanIQ_portfolio = "Ph1_LoanIQ"
ph2_loanIQ_portfolio = "Ph2_LoanIQ"
ph3_loanIQ_portfolio = "Ph3_LoanIQ"

simple_instrument_scope = "simple_bdh"
full_instrument_scope = "full_bdh"

create_portfolio (ph1_murex_portfolio, murex_scope, simple_instrument_scope)
create_portfolio (ph2_murex_portfolio, murex_scope, full_instrument_scope)
create_portfolio (ph3_murex_portfolio, sdm_scope, full_instrument_scope)

create_portfolio (ph1_loanIQ_portfolio, loanIQ_scope, simple_instrument_scope)
create_portfolio (ph2_loanIQ_portfolio, loanIQ_scope, full_instrument_scope)
create_portfolio (ph3_loanIQ_portfolio, sdm_scope, full_instrument_scope)


{"name":"PortfolioWithIdAlreadyExists","errorDetails":[],"code":112,"type":"https://docs.lusid.com/#section/Error-Codes/112","title":"Could not create a portfolio with id 'Ph1_Murex' because it already exists in scope 'bdh-murex'.","status":400,"detail":"Error creating portfolio with id 'Ph1_Murex' in scope 'bdh-murex' effective at 2010-01-01T00:00:00.0000000+00:00 because it already exists.","instance":"https://calculation.lusid.com/app/insights/logs/0HMKNRVOM69EJ:00000027","extensions":{}}
{"name":"PortfolioWithIdAlreadyExists","errorDetails":[],"code":112,"type":"https://docs.lusid.com/#section/Error-Codes/112","title":"Could not create a portfolio with id 'Ph2_Murex' because it already exists in scope 'bdh-murex'.","status":400,"detail":"Error creating portfolio with id 'Ph2_Murex' in scope 'bdh-murex' effective at 2010-01-01T00:00:00.0000000+00:00 because it already exists.","instance":"https://calculation.lusid.com/app/insights/logs/0HMKNRM31UTPK:00000024","extensions":{}}
{"name

# 3. Phase I: Data load

Using Source System datamodel and LUSID Simple Instruments

In [17]:
# Function to create Simple Instruments for the Phase 1 example
def create_simple_instrument(instrument_scope, currency, maturity_date, name, identifier):
    
    simple_instrument = lm.SimpleInstrument(
        dom_ccy = currency,
        maturity_date = maturity_date,
        asset_class = "Money",
        instrument_type = "SimpleInstrument",
        simple_instrument_type = "Loan"
    )

    instrument_definition = lm.InstrumentDefinition(
        name=name,
        identifiers={"ClientInternal" : lm.InstrumentIdValue(value=identifier)},
        definition = simple_instrument,
    )

    # upsert the instrument
    upsert_request = {identifier: instrument_definition}
    upsert_response = instruments_api.upsert_instruments(
        request_body=upsert_request,
        scope = instrument_scope)
    
    instrument_luid = upsert_response.values[identifier].lusid_instrument_id
    print(instrument_luid)

In [18]:
# Function to book the loan transactions and simple instruments
def book_simple_transaction(portfolio_code, portfolio_scope, deal_id, deal_identifier, trade_date, currency, notional, properties):
    
    # standard 2 day settlement
    settlement_date = trade_date + timedelta(days = 2)
    
    trade_txn = lm.TransactionRequest(
        transaction_id= deal_id + ".1",
        type="Buy",
        instrument_identifiers={"Instrument/default/ClientInternal": deal_identifier},
        transaction_date=trade_date,
        settlement_date=settlement_date,
        units=notional,
        transaction_price=lm.TransactionPrice(price=100,type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=notional ,currency=currency),
        exchange_rate=1,
        source = portfolio_scope,
        transaction_currency = currency,
        properties = properties
    )
    
    response = transaction_portfolios_api.upsert_transactions(scope=portfolio_scope,
                                                        code=portfolio_code,
                                                        transaction_request=[trade_txn])

    print(f"Transaction successfully updated at time: {response.version.as_at_date}")

In [19]:
def create_and_upsert_simple_instrument_transactions(model_key, deal_key, portfolio, portfolio_scope, trade_df, trade_header_df):

    # main loop to run through the worksheet and upload the transactions
    identifier_type = "ClientInternal"

    for index, row in trade_df.iterrows():
        deal_id = deal_key + str(index)

        properties = {}

        for index1, header_row in trade_header_df.iterrows():
                key = header_row[model_key]
                value = row[header_row['Name']]
                data_type = header_row['Type']

                if not (value == "" or pd.isnull(value)):
                    if data_type == "string":
                        properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(label_value = value)
                        )
                    elif data_type == "date":
                        properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(label_value = datetime.strptime(value, '%d/%m/%Y').replace(tzinfo=pytz.utc))
                        )
                    elif data_type == "number":
                         properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(metric_value = lm.MetricValue(value = value))
                        )
                    else:
                        print(f"Could not add property with Key: {key} Value: {value} Type: {data_type}")

        client_internal_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'clientInternal'].reset_index()['Name'][0]
        trade_date_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'transactionDate'].reset_index()['Name'][0]
        maturity_date_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'maturityDate'].reset_index()['Name'][0]
        product_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'product'].reset_index()['Name'][0]
        product = row[product_key]

        # As Murex has forwards we need to do some bespoke mappings here
        if product == "Depo" or product == "CommercialLoan":
            transaction_currency_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'transactionCurrency'].reset_index()['Name'][0]
            units_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'notional'].reset_index()['Name'][0]
        elif product == "Forward":
            transaction_currency_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'domCcy'].reset_index()['Name'][0]
            units_key = trade_header_df[trade_header_df['SimpleInstMap'] == 'domNotional'].reset_index()['Name'][0]

        # create and upload the instrument
        instrument_name = row[client_internal_key]
        instrument_identifier = row[client_internal_key]
        maturity_date = datetime.strptime(row[maturity_date_key], '%d/%m/%Y').replace(tzinfo=pytz.utc)
        trade_date = datetime.strptime(row[trade_date_key], '%d/%m/%Y').replace(tzinfo=pytz.utc)

        # create and load the instrument
        create_simple_instrument (simple_instrument_scope, row[transaction_currency_key], maturity_date, instrument_name, instrument_identifier)

        response = book_simple_transaction(portfolio, portfolio_scope, deal_id, row[client_internal_key], trade_date, row[transaction_currency_key], row[units_key], properties)
        

In [20]:
# Load the trades in for both Murex and LoanIQ

# Read Murex trade data
murex_df = pd.read_csv("data/Murex-TradeData.csv")
murex_df.head()

,Name,dealReference,description,dealGroup,account,product,portfolio,desk,partyReference,tradeDate,maturityDate,expiryDate,ccy,notional,domCcy,domNotional,fgnCcy,fgnNotional,interestRate,forwardRate,dayBasis,ratesRollConvention,sourceSystem
0,1,LN044336001,Deposit / Loan,MM,LDN-MM-001,Depo,LN-Corp-Sales,MM,AMZL,20/05/2021,20/05/2022,NaN,EUR,"1,000,000.0000",NaN,NaN,NaN,NaN,4.0000,NaN,Exact/360,Following,Murex
1,2,LN044347002,Deposit / Loan,MM,LDN-MM-001,Depo,LN-Corp-Sales,MM,AAPL,31/05/2021,31/05/2022,NaN,USD,"2,000,000.0000",NaN,NaN,NaN,NaN,5.0000,NaN,Exact/360,Following,Murex
2,3,LN044353003,FX Forward,FX,LDN-FX-Sales-JP,Forward,LN-Hedge-FX,FX,META,06/06/2021,11/06/2021,NaN,NaN,NaN,EUR,"15,000,000.0000",USD,"17,300,000.0000",NaN,1.1500,NaN,NaN,Murex
3,4,LN044384004,FX Forward,FX,LDN-FX-Sales-JP,Forward,LN-Hedge-FX,FX,META,07/07/2021,12/07/2021,NaN,NaN,NaN,EUR,"-20,000,000.0000",USD,"-224,000,000.0000",NaN,1.1200,NaN,NaN,Murex
4,5,LN044384005,Deposit / Loan,MM,LDN-MM-002,Depo,LN-Corp-Sales2,MM,GOOG,07/07/2021,03/01/2022,NaN,EUR,"-3,000,000.0000",NaN,NaN,NaN,NaN,3.5000,NaN,Exact/360,Following,Murex


In [21]:
# Read LoanIQ trade data
loanIQ_df = pd.read_csv("data/LoanIQ-TradeData.csv")
loanIQ_df.head()

,Name,balanceSheetDate,contractReference,contractDescription,contractGroup,accountReference,productType,book,signatureDate,originDate,maturityDate,currency,principal,basis,fixedRate,interestRateType,rateCap,rateFloor,rollConvention,businessRollDayConvention,sourceSystem
0,1,01/05/2021,05010PTY00100,Loans with Fixed Principal + Accrued Interest ...,05010DEFAULTEUR,01173b439acd7B2j,CommercialLoan,LN-Com-Loans1,19/02/2021,19/02/2021,19/02/2022,EUR,100000,Exact/360,8,Fixed,99.9900,0,NaN,Following,LoanIQ
1,2,01/05/2021,0205010PTY00100,Loans with Fixed Principal + Accrued Interest ...,05010DEFAULTEUR,011734c0c05b6TRX,CommercialLoan,LN-Com-Loans2,14/07/2020,14/07/2020,14/10/2021,EUR,300000,Exact/360,8,Fixed,99.9900,0,NaN,Following,LoanIQ
2,3,01/05/2021,0105010PTY00101,Loans with Fixed Principal + Accrued Interest ...,05010DEFAULTUSD,011734893e4165Ef,CommercialLoan,LN-Com-Loans1,13/07/2020,13/07/2020,13/07/2022,USD,300000,Exact/360,8,Fixed,99.9900,0,NaN,Following,LoanIQ
3,4,01/05/2021,0105010PTY00104,Loans with Fixed Principal + Accrued Interest ...,05010DEFAULTUSD,011734db06940nOu,CommercialLoan,LN-Com-Loans2,14/10/2020,14/10/2020,14/04/2022,USD,200000,Exact/360,4,Variable,99.9900,0,NaN,Following,LoanIQ
4,5,01/05/2021,0105010PTY00103,Loans with Fixed Principal + Accrued Interest ...,05010DEFAULTUSD,0117348aa06e16tx,CommercialLoan,LN-Com-Loans3,13/07/2020,13/07/2020,13/07/2022,USD,300000,Exact/360,10,Variable,99.9900,0,NaN,Following,LoanIQ


In [22]:
# load the Murex data for Phase I
create_and_upsert_simple_instrument_transactions("Key", "TXNM001", ph1_murex_portfolio, murex_scope, murex_df, murex_header_df)

# Load the LoanIQ data for Phase I
create_and_upsert_simple_instrument_transactions("Key", "TXNL001", ph1_loanIQ_portfolio, loanIQ_scope, loanIQ_df, loanIQ_header_df)


LUID_000049DT
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DU
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DV
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DW
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DX
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DY
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049DZ
Transaction successfully updated at time: 2022-09-15 13:34:57.083932+00:00
LUID_000049E0
Transaction successfully updated at time: 2022-09-15 13:34:57.796093+00:00
LUID_000049E1
Transaction successfully updated at time: 2022-09-15 13:34:57.796093+00:00
LUID_000049E2
Transaction successfully updated at time: 2022-09-15 13:34:57.796093+00:00
LUID_000049E3
Transaction successfully updated at time: 2022-09-15 13:34:57.796093+00:00
LUID_000049E4
Transac

# 4. Phase II. Data load

Using Source System datamodel and LUSID Full Instruments

In [23]:
# Now add the transactions to Phase 2 portfolio, for this we model the instruments in LUSID as simple Bond Instruments
def create_termDeposit(instrument_scope, currency, roll_convention, day_count_convention, start_date, maturity_date, coupon_rate, instrument_name, instrument_identifier):
    
    flow_convention = lm.FlowConventions(
        currency=currency,
        payment_frequency = "0Invalid",
        roll_convention=roll_convention,
        day_count_convention=day_count_convention,
        payment_calendars=[],
        reset_calendars=[],
        settle_days=0,
        reset_days=0,
    )

    termDeposit_definition = lm.TermDeposit(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_ccy=currency,
        contract_size=1,
        rate=float(coupon_rate)/100,
        flow_convention=flow_convention,
        instrument_type="TermDeposit",
    )

    # define the instrument to be upserted
    instrument_definition = lm.InstrumentDefinition(
        name=instrument_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(instrument_identifier)},
        definition=termDeposit_definition,
    )
    
    # upsert the instrument
    upsert_request = {instrument_identifier: instrument_definition}
    upsert_response = instruments_api.upsert_instruments(
        request_body=upsert_request,
        scope = instrument_scope)
    
    instrument_luid = upsert_response.values[instrument_identifier].lusid_instrument_id
    print(instrument_luid)

In [24]:
def create_fxForward(instrument_scope, dom_ccy, fgn_ccy, dom_amount, fgn_amount, start_date, maturity_date, instrument_name, instrument_identifier):
    
    fxForward_definition = lm.FxForward(
        start_date=start_date,
        maturity_date=maturity_date,
        dom_amount=1,
        dom_ccy=dom_ccy,
        fgn_amount=-1 * float(fgn_amount) / float(dom_amount),
        fgn_ccy=fgn_ccy,
        instrument_type="FxForward"
    )
    
    # define the instrument to be upserted
    instrument_definition = lm.InstrumentDefinition(
        name=instrument_name,
        identifiers={"ClientInternal": lm.InstrumentIdValue(instrument_identifier)},
        definition=fxForward_definition,
    )
    
    # upsert the instrument
    upsert_request = {instrument_identifier: instrument_definition}
    upsert_response = instruments_api.upsert_instruments(
        request_body=upsert_request,
        scope = instrument_scope)
    
    instrument_luid = upsert_response.values[instrument_identifier].lusid_instrument_id
    print(instrument_luid)    

In [25]:
# Function to book the FXForward transactions seen in Murex (main difference is that this is a StockIn, with no consideration)
def book_fxForward_transaction(portfolio_code, portfolio_scope, deal_id, deal_identifier, trade_date, currency, notional, properties):
    
    # standard 2 day settlement
    settlement_date = trade_date + timedelta(days = 2)
    
    trade_txn = lm.TransactionRequest(
        transaction_id= deal_id + ".1",
        type="StockIn",
        instrument_identifiers={"Instrument/default/ClientInternal": deal_identifier},
        transaction_date=trade_date,
        settlement_date=settlement_date,
        units=notional,
        transaction_price=lm.TransactionPrice(price=0,type="Price"),
        total_consideration=lm.CurrencyAndAmount(amount=0 ,currency=currency),
        exchange_rate=1,
        source = portfolio_scope,
        transaction_currency = currency,
        properties = properties
    )
    
    response = transaction_portfolios_api.upsert_transactions(scope=portfolio_scope,
                                                        code=portfolio_code,
                                                        transaction_request=[trade_txn])

    print(f"Transaction successfully updated at time: {response.version.as_at_date}")

In [26]:
def create_and_upsert_full_instrument_transactions(model_key, deal_key, portfolio, portfolio_scope, trade_df, trade_header_df):

    # main loop to run through the worksheet and upload the transactions
    identifier_type = "ClientInternal"

    for index, row in trade_df.iterrows():
        deal_id = deal_key + str(index)

        properties = {}

        for index1, header_row in trade_header_df.iterrows():
                key = header_row[model_key]
                value = row[header_row['Name']]
                data_type = header_row['Type']

                if not (value == "" or pd.isnull(value)):
                    if data_type == "string":
                        properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(label_value = value)
                        )
                    elif data_type == "date":
                        properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(label_value = datetime.strptime(value, '%d/%m/%Y').replace(tzinfo=pytz.utc))
                        )
                    elif data_type == "number":
                         properties[key] = lm.PerpetualProperty(
                            key = key,
                            value = lm.PropertyValue(metric_value = lm.MetricValue(value = value))
                        )
                    else:
                        print(f"Could not add property with Key: {key} Value: {value} Type: {data_type}")

        client_internal_key = trade_header_df[trade_header_df['FullInstMap'] == 'clientInternal'].reset_index()['Name'][0]
        trade_date_key = trade_header_df[trade_header_df['FullInstMap'] == 'transactionDate'].reset_index()['Name'][0]
        maturity_date_key = trade_header_df[trade_header_df['FullInstMap'] == 'maturityDate'].reset_index()['Name'][0]
        product_key = trade_header_df[trade_header_df['FullInstMap'] == 'product'].reset_index()['Name'][0]

        instrument_name = row[client_internal_key]
        instrument_identifier = row[client_internal_key]
        maturity_date = datetime.strptime(row[maturity_date_key], '%d/%m/%Y').replace(tzinfo=pytz.utc)
        trade_date = datetime.strptime(row[trade_date_key], '%d/%m/%Y').replace(tzinfo=pytz.utc)
        product = row[product_key]

        # As Murex has forwards we need to do some bespoke mappings here
        if product == "Depo" or product == "CommercialLoan":
            transaction_currency_key = trade_header_df[trade_header_df['FullInstMap'] == 'transactionCurrency'].reset_index()['Name'][0]
            units_key = trade_header_df[trade_header_df['FullInstMap'] == 'notional'].reset_index()['Name'][0]
            day_count_convention_key = trade_header_df[trade_header_df['FullInstMap'] == 'dayCountConvention'].reset_index()['Name'][0]

            roll_convention_key = trade_header_df[trade_header_df['FullInstMap'] == 'rollConvention'].reset_index()['Name'][0]
            fixed_rate_key = trade_header_df[trade_header_df['FullInstMap'] == 'fixedRate'].reset_index()['Name'][0]
            day_count_convention = row[day_count_convention_key] 
            if day_count_convention == "Exact/360":
                day_count_convention = "Actual360"

            # create and upload the instrument
            create_termDeposit(full_instrument_scope, row[transaction_currency_key], row[roll_convention_key], day_count_convention, trade_date, maturity_date, row[fixed_rate_key], instrument_name, instrument_identifier)

            # create the trade booking
            response = book_simple_transaction(portfolio, portfolio_scope, deal_id, row[client_internal_key], trade_date, row[transaction_currency_key], row[units_key], properties)

        elif product == "Forward":
            dom_currency_key = trade_header_df[trade_header_df['FullInstMap'] == 'domCcy'].reset_index()['Name'][0]
            fgn_currency_key = trade_header_df[trade_header_df['FullInstMap'] == 'fgnCcy'].reset_index()['Name'][0]
            dom_amount_key = trade_header_df[trade_header_df['FullInstMap'] == 'domNotional'].reset_index()['Name'][0]
            fgn_amount_key = trade_header_df[trade_header_df['FullInstMap'] == 'fgnNotional'].reset_index()['Name'][0]
            

            # create and upload the instrument
            create_fxForward(full_instrument_scope, row[dom_currency_key], row[fgn_currency_key], row[dom_amount_key], row[fgn_amount_key], trade_date, maturity_date, instrument_name, instrument_identifier)

            # create the trade booking
            response = book_fxForward_transaction(portfolio, portfolio_scope, deal_id, row[client_internal_key], trade_date, row[dom_currency_key], row[dom_amount_key], properties)
            
        

In [27]:
#load the Ph2 portfolio transactions for Murex
create_and_upsert_full_instrument_transactions("Key", "TXNM002", ph2_murex_portfolio, murex_scope, murex_df, murex_header_df)

#load the Ph2 portfolio transactions for LoanIQ
create_and_upsert_full_instrument_transactions("Key", "TXNL002", ph2_loanIQ_portfolio, loanIQ_scope, loanIQ_df, loanIQ_header_df)


LUID_000049E7
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049E8
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049E9
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049EA
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049EB
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049EC
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049ED
Transaction successfully updated at time: 2022-09-15 13:34:57.320198+00:00
LUID_000049EE
Transaction successfully updated at time: 2022-09-15 13:34:58.063205+00:00
LUID_000049EF
Transaction successfully updated at time: 2022-09-15 13:34:58.063205+00:00
LUID_000049EG
Transaction successfully updated at time: 2022-09-15 13:34:58.063205+00:00
LUID_000049EH
Transaction successfully updated at time: 2022-09-15 13:34:58.063205+00:00
LUID_000049EI
Transac

In [28]:
# now add the derived properties

# These are the derived properties for Phase 2, so SDM is derived from the LoanIQ key in the murex scope
property_keys = add_derived_property_keys (loanIQ_scope, "SDMMapping", "Type", "Name", loanIQ_header_df)

# These are the derived properties for Phase 2, so SDM is derived from the Murex key in the murex scope
property_keys = add_derived_property_keys (murex_scope, "SDMMapping", "Type", "Name", murex_header_df)

Property definition with the following key already exists: Transaction/bdh-loanIQ/balanceSheetDate
Property definition created with the following key: Transaction/bdh-loanIQ/tradeReference
Property definition created with the following key: Transaction/bdh-loanIQ/tradeDescription
Property definition created with the following key: Transaction/bdh-loanIQ/tradeGroup
Property definition with the following key already exists: Transaction/bdh-loanIQ/accountReference
Property definition with the following key already exists: Transaction/bdh-loanIQ/productType
Property definition created with the following key: Transaction/bdh-loanIQ/portfolio
Property definition created with the following key: Transaction/bdh-loanIQ/confirmationDate
Property definition created with the following key: Transaction/bdh-loanIQ/tradeDate
Property definition with the following key already exists: Transaction/bdh-loanIQ/maturityDate
Property definition with the following key already exists: Transaction/bdh-loanIQ/c

# 5. Phase III. Data load

Using SDM datamodel and LUSID Full Instruments. We simply re-use the function which was used for Phase II, but change the data mapping from the source system to the canonical SDM mapping, which is held using the SDMKey.

In [29]:

#load the Ph2 portfolio transactions for Murex
create_and_upsert_full_instrument_transactions("SDMKey", "TXNM003", ph3_murex_portfolio, sdm_scope, murex_df, murex_header_df)

#load the Ph2 portfolio transactions for LoanIQ
create_and_upsert_full_instrument_transactions("SDMKey", "TXNL003", ph3_loanIQ_portfolio, sdm_scope, loanIQ_df, loanIQ_header_df)



LUID_000049E7
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049E8
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049E9
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049EA
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049EB
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049EC
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049ED
Transaction successfully updated at time: 2022-09-15 13:34:57.560901+00:00
LUID_000049EE
Transaction successfully updated at time: 2022-09-15 13:34:58.317259+00:00
LUID_000049EF
Transaction successfully updated at time: 2022-09-15 13:34:58.317259+00:00
LUID_000049EG
Transaction successfully updated at time: 2022-09-15 13:34:58.317259+00:00
LUID_000049EH
Transaction successfully updated at time: 2022-09-15 13:34:58.317259+00:00
LUID_000049EI
Transac

In [30]:
# Create derived properties representing the source system data model, in the Ph3 portfolios so we can query the data in the old format

# These are the derived properties for Phase 3, so LoanIQ is derived from the SDM key in the sdm scope
property_keys = add_derived_property_keys (sdm_scope, "Name", "Type", "SDMMapping", loanIQ_header_df)

# These are the derived properties for Phase 3, so Murex is derived from the SDM key in the sdm scope
property_keys = add_derived_property_keys (sdm_scope, "Name", "Type", "SDMMapping", murex_header_df)

Property definition with the following key already exists: Transaction/bdh-sdm/balanceSheetDate
Property definition created with the following key: Transaction/bdh-sdm/contractReference
Property definition created with the following key: Transaction/bdh-sdm/contractDescription
Property definition created with the following key: Transaction/bdh-sdm/contractGroup
Property definition with the following key already exists: Transaction/bdh-sdm/accountReference
Property definition with the following key already exists: Transaction/bdh-sdm/productType
Property definition created with the following key: Transaction/bdh-sdm/book
Property definition created with the following key: Transaction/bdh-sdm/signatureDate
Property definition created with the following key: Transaction/bdh-sdm/originDate
Property definition with the following key already exists: Transaction/bdh-sdm/maturityDate
Property definition with the following key already exists: Transaction/bdh-sdm/currency
Property definition cre

# 5. FX Market Data

Market data is loaded into LUSID to be able to value the loans and FxForward trades, the market data comprises:

1. EURUSD Spot rates

2. EUR and USD discount curves

## 5.1 FX Spot Rates 

In [31]:
# Read fx spot rates and make datetimes timezone aware
quotes_df = pd.read_csv("data/eurusd_spot.csv")
quotes_df["Date"] = pd.to_datetime(quotes_df["Date"], dayfirst=True)
quotes_df["Date"] = quotes_df["Date"].apply(lambda x: x.replace(tzinfo=pytz.utc))
quotes_df.head()

,Date,Rate,Pair
0,2020-07-13 00:00:00+00:00,1.2215,EUR/USD
1,2020-07-14 00:00:00+00:00,1.2248,EUR/USD
2,2020-07-15 00:00:00+00:00,1.2298,EUR/USD
3,2020-07-16 00:00:00+00:00,1.2327,EUR/USD
4,2020-07-17 00:00:00+00:00,1.2272,EUR/USD


In [32]:
# Create quotes request
instrument_quotes = {
            index: lm.UpsertQuoteRequest(
            quote_id=lm.QuoteId(
                quote_series_id=lm.QuoteSeriesId(
                    provider=market_supplier,
                    instrument_id=row["Pair"],
                    instrument_id_type="CurrencyPair",
                    quote_type="Rate",
                    field="mid",
                ),
                effective_at=row["Date"],
            ),
            metric_value=lm.MetricValue(value=row["Rate"], unit=row["Pair"]),
        )
    for index, row in quotes_df.iterrows()
}

# Upsert quotes into LUSID
response = quotes_api.upsert_quotes(
    scope=market_data_scope, request_body=instrument_quotes
)

if response.failed == {}:
    print(f"Quotes successfully loaded into LUSID. {len(response.values)} quotes loaded.")
else:
    print(f"Some failures occurred during quotes upsertion, {len(response.failed)} did not get loaded into LUSID.")

Quotes successfully loaded into LUSID. 739 quotes loaded.


## 5.2 EUR & USD Discount curves
Next we will read in the EUR and USD Discount factors and create separate OIS curves.

In [33]:
# This is the earliest date we have loans in the portfolio
base_date = datetime(2020, 7, 13, tzinfo=pytz.utc)

# Read FX market data - showing first 10 terms
df_market_data = pd.read_csv("data/EURUSD-MarketData.csv")
df_market_data["Days"]= df_market_data["Days"].astype(int)
df_market_data = df_market_data.reset_index()
df_market_data.head(10)

,index,Tenor,Days,EUR-Rates,USD-Rates,EURdf,USDdf,Vol25D,ATM,Vol75D,Strike25D,StrikeATM,Strike75D
0,0,1M,30,0.2200,1.2200,0.9998,0.9990,0.1200,0.1000,0.1100,1.1850,1.2000,1.2150
1,1,2M,60,0.3700,1.3700,0.9994,0.9978,0.1205,0.1010,0.1105,1.1850,1.2000,1.2150
2,2,3M,90,0.5300,1.5300,0.9987,0.9962,0.1210,0.1020,0.1110,1.1850,1.2000,1.2150
3,3,6M,180,0.9500,1.9500,0.9953,0.9904,0.1215,0.1030,0.1115,1.1850,1.2000,1.2150
4,4,1Y,365,1.3000,2.3000,0.9871,0.9773,0.1220,0.1040,0.1120,1.1850,1.2000,1.2150
5,5,18M,547,1.5000,2.5000,0.9778,0.9632,0.1225,0.1050,0.1125,1.1850,1.2000,1.2150
6,6,2Y,730,1.5000,2.5000,0.9705,0.9513,0.1230,0.1060,0.1130,1.1850,1.2000,1.2150
7,7,3Y,1095,1.4300,2.4300,0.9580,0.9297,0.1235,0.1070,0.1135,1.1850,1.2000,1.2150
8,8,4Y,1460,1.3700,2.3700,0.9467,0.9096,0.1240,0.1080,0.1140,1.1850,1.2000,1.2150
9,9,5Y,1825,1.3100,2.3100,0.9366,0.8909,0.1245,0.1090,0.1145,1.1850,1.2000,1.2150


In [34]:
# Set up the OIS curves loader function
def upsert_discount_factors(scope, effective_at, market_asset, dates, dfs):

    complex_market_data = lm.DiscountFactorCurveData(
        base_date=effective_at,
        dates=dates,
        discount_factors = dfs,
        market_data_type="DiscountFactorCurveData"
    )

    # create a unique identifier for our OIS yield curves
    complex_id = lm.ComplexMarketDataId(
        provider=market_supplier,
        price_source = market_supplier,
        effective_at=effective_at,
        market_asset=market_asset,
    )
    
    upsert_request = lm.UpsertComplexMarketDataRequest(
        market_data_id=complex_id, market_data=complex_market_data
    )


    response = complex_market_data_api.upsert_complex_market_data(
        scope=scope, request_body={market_asset: upsert_request}
    )
    
    if response.failed:
        print(f"Failed to upload yield curve {response.failed}")

    print(f"{market_asset} yield curve uploaded into scope={scope}")
    
    return complex_id

In [35]:
# Set up the OIS curves
dates = []
dfs = []

# loading the data for EUR OIS discount curve
for index, row in df_market_data.iterrows():
    dates.append(base_date + timedelta(days=row["Days"]))
    dfs.append(row["EURdf"])

curve_id_EUR = upsert_discount_factors(market_data_scope, base_date, "EUR/EUROIS", dates, dfs)

dates = []
dfs = []

# loading the data for USD OIS discount curve
for index, row in df_market_data.iterrows():
    dates.append(base_date + timedelta(days=row["Days"]))
    dfs.append(row["USDdf"])
    
curve_id_USD = upsert_discount_factors(market_data_scope, base_date, "USD/USDOIS", dates, dfs)

EUR/EUROIS yield curve uploaded into scope=bdh
USD/USDOIS yield curve uploaded into scope=bdh


# 6. Valuation

This sections sets up the model recipes for the valuations, and demonstrates so valuations at different points in the portfolio lifecycle

In [36]:
# Set recipe code
recipe_code = "SimpleValuation"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                    key="FX.CurrencyPair.*",
                    supplier=market_supplier,
                    data_scope=market_data_scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="2Y"
                ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier=market_supplier,
                data_scope=market_data_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="2Y",
            ),
        ],
        options=lm.MarketOptions(
            default_scope = market_data_scope,
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="TermDeposit",
                parameters="{}",
            ),
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="FxForward",
                parameters="{}",
            )
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation = True
        )
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2022-09-16 12:40:37.894790+00:00.


In [37]:
# Set recipe code
qps_recipe_code = "QPS-Valuation"

# Populate recipe parameters
configuration_recipe = lm.ConfigurationRecipe(
    scope=market_data_scope,
    code=qps_recipe_code,
    market=lm.MarketContext(
        market_rules=[
            lm.MarketDataKeyRule(
                    key="FX.CurrencyPair.*",
                    supplier=market_supplier,
                    data_scope=market_data_scope,
                    quote_type="Rate",
                    field="mid",
                    quote_interval="2Y"
                ),
            lm.MarketDataKeyRule(
                key="Rates.*.*",
                supplier=market_supplier,
                data_scope=market_data_scope,
                price_source=market_supplier,
                quote_type="Price",
                field="mid",
                quote_interval="2Y",
            ),
        ],
        options=lm.MarketOptions(
            default_scope = market_data_scope,
            attempt_to_infer_missing_fx=True
        ),
    ),
    pricing=lm.PricingContext(
        model_rules=[
            lm.VendorModelRule(
                supplier="RefinitivQps",
                model_name="VendorDefault",
                instrument_type="TermDeposit",
                parameters="{}",
            ),
            lm.VendorModelRule(
                supplier="RefinitivQps",
                model_name="VendorDefault",
                instrument_type="FxForward",
                parameters="{}",
            ),
            lm.VendorModelRule(
                supplier="Lusid",
                model_name="Discounting",
                instrument_type="CashSettled",
                parameters="{}",
            ),
        ],
        options = lm.PricingOptions(
            allow_partially_successful_evaluation = True
        )
    ),
)

response = configuration_recipe_api.upsert_configuration_recipe(
    upsert_recipe_request=lm.UpsertRecipeRequest(
        configuration_recipe=configuration_recipe
    )
)


print(f"Configuration recipe loaded into LUSID at time {response.value}.")

Configuration recipe loaded into LUSID at time 2022-09-16 12:40:38.074555+00:00.


In [43]:
def run_valuation(date, portfolio_entities, recipe_code):

    metrics = [
        lm.AggregateSpec("Instrument/default/Name", "Value"),
        lm.AggregateSpec("Holding/Units", "Value"),
        lm.AggregateSpec("Holding/default/PV", "Value"),
        lm.AggregateSpec("Holding/default/Units", "Value"),
        lm.AggregateSpec("Valuation/PvInPortfolioCcy", "Value"),
        lm.AggregateSpec("Holding/default/Currency", "Value"),  
        lm.AggregateSpec("Portfolio/default/Name", "Value"),  
    ]

    group_by = ""
    #group_by = ["Portfolio/default/Name"]
    #group_by = ["Portfolio/default/Name", "Holding/default/Currency"]
    
    
    portfolio_entities = portfolio_entities
    
    valuation_request = lm.ValuationRequest(
        recipe_id=lm.ResourceId(scope=market_data_scope, code=recipe_code),
        metrics=metrics,
        group_by=group_by,
        portfolio_entity_ids=portfolio_entities,
        valuation_schedule=lm.ValuationSchedule(effective_at=date),
    )

    val = aggregation_api.get_valuation(valuation_request=valuation_request)
    #display(val)
    val_data = val.data
    
    vals_df = pd.DataFrame(val_data)

    return vals_df

In [44]:
portfolio_entities = []

#portfolio_entities.append(lm.PortfolioEntityId(scope=murex_scope, code=ph1_murex_portfolio))
#portfolio_entities.append(lm.PortfolioEntityId(scope=loanIQ_scope, code=ph1_loanIQ_portfolio))
#portfolio_entities.append(lm.PortfolioEntityId(scope=murex_scope, code=ph2_murex_portfolio))
#portfolio_entities.append(lm.PortfolioEntityId(scope=loanIQ_scope, code=ph2_loanIQ_portfolio))
portfolio_entities.append(lm.PortfolioEntityId(scope=sdm_scope, code=ph3_murex_portfolio))
portfolio_entities.append(lm.PortfolioEntityId(scope=sdm_scope, code=ph3_loanIQ_portfolio))

valuation_date = datetime(2021, 6, 11, tzinfo=pytz.utc)
valuation = run_valuation(valuation_date, portfolio_entities, recipe_code)
display(valuation)

,Instrument/default/Name,Holding/Units,Holding/default/PV,Holding/default/Units,Valuation/PvInPortfolioCcy,Holding/default/Currency,Portfolio/default/Name
0,LN044336001,"1,000,000.0000","1,023,855.1898","1,000,000.0000","1,023,855.1898",EUR,Ph3_Murex
1,EUR,"-1,000,000.0000","-1,000,000.0000","-1,000,000.0000","-1,000,000.0000",EUR,Ph3_Murex
2,LN044347002,"2,000,000.0000","1,718,367.9368","2,000,000.0000","1,718,367.9368",USD,Ph3_Murex
3,USD,"-2,000,000.0000","-1,679,120.1410","-2,000,000.0000","-1,679,120.1410",USD,Ph3_Murex
4,LN044353003,"15,000,000.0000","475,610.7800","15,000,000.0000","475,610.7800",EUR,Ph3_Murex
5,0105010PTY00101,"300,000.0000","284,283.4299","300,000.0000","284,283.4299",USD,Ph3_LoanIQ
6,0105010PTY00103,"300,000.0000","294,203.4540","300,000.0000","294,203.4540",USD,Ph3_LoanIQ
7,0205010PTY00100,"300,000.0000","328,401.7456","300,000.0000","328,401.7456",EUR,Ph3_LoanIQ
8,USD,"-2,000,000.0000","-1,679,120.1410","-2,000,000.0000","-1,679,120.1410",USD,Ph3_LoanIQ
9,EUR,"-400,000.0000","-400,000.0000","-400,000.0000","-400,000.0000",EUR,Ph3_LoanIQ


In [45]:
# The following valuation requires licenses for the three external services; only refinitiv should have the QPS licence.
qps_valuation = run_valuation(valuation_date, portfolio_entities, qps_recipe_code)
pd.DataFrame(qps_valuation)

,Instrument/default/Name,Holding/Units,Holding/default/PV,Holding/default/Units,Valuation/PvInPortfolioCcy,Holding/default/Currency,Portfolio/default/Name
0,LN044336001,"1,000,000.0000","1,045,859.9521","1,000,000.0000","1,045,859.9521",EUR,Ph3_Murex
1,EUR,"-1,000,000.0000","-1,000,000.0000","-1,000,000.0000","-1,000,000.0000",EUR,Ph3_Murex
2,LN044347002,"2,000,000.0000","1,761,792.1753","2,000,000.0000","1,761,792.1753",USD,Ph3_Murex
3,USD,"-2,000,000.0000","-1,679,120.1410","-2,000,000.0000","-1,679,120.1410",USD,Ph3_Murex
4,LN044353003,"15,000,000.0000","710,976.4187","15,000,000.0000","710,976.4187",EUR,Ph3_Murex
5,0105010PTY00101,"300,000.0000","292,260.6157","300,000.0000","292,260.6157",USD,Ph3_LoanIQ
6,0105010PTY00103,"300,000.0000","302,459.0024","300,000.0000","302,459.0024",USD,Ph3_LoanIQ
7,0205010PTY00100,"300,000.0000","331,079.6317","300,000.0000","331,079.6317",EUR,Ph3_LoanIQ
8,USD,"-2,000,000.0000","-1,679,120.1410","-2,000,000.0000","-1,679,120.1410",USD,Ph3_LoanIQ
9,EUR,"-400,000.0000","-400,000.0000","-400,000.0000","-400,000.0000",EUR,Ph3_LoanIQ


# 7. Trade Lifecycle

In [41]:
# Example as to how to get transactions 
cash_flows = transaction_portfolios_api.get_upsertable_portfolio_cash_flows(
    scope=sdm_scope,
    code=ph3_murex_portfolio,
    effective_at=valuation_date,
    window_start=base_date,
    window_end=valuation_date,
    recipe_id_scope=market_data_scope,
    recipe_id_code=recipe_code
)

# we create a dataframe out of the cash flows table
cash_flow_table = lusid_response_to_data_frame(cash_flows)
cash_flow_table

,transaction_id,type,instrument_identifiers.Instrument/default/LusidInstrumentId,instrument_scope,instrument_uid,transaction_date,settlement_date,units,transaction_price.price,transaction_price.type,total_consideration.amount,total_consideration.currency,exchange_rate,transaction_currency,properties,source,entry_date_time,transaction_status
0,TXNM0032.1-LUID_000049E9-20210611-Principal-EU...,CashFlow,LUID_000049E9,full_bdh,LUID_000049E9,2021-06-11 00:00:00+00:00,2021-06-11 00:00:00+00:00,"15,000,000.0000",1.0000,Price,"15,000,000.0000",EUR,1.0000,EUR,{},default,0001-01-01 00:00:00+00:00,Active
1,TXNM0032.1-LUID_000049E9-20210611-Principal-US...,CashFlow,LUID_000049E9,full_bdh,LUID_000049E9,2021-06-11 00:00:00+00:00,2021-06-11 00:00:00+00:00,"-17,300,000.0000",1.0000,Price,"-17,300,000.0000",USD,1.0000,USD,{},default,0001-01-01 00:00:00+00:00,Active
